#  AWS Machine Learning Engineer Nanodegree Scholarship – Udacity 
# Optimizing Starbucks rewards using Machine Learning 
## [Capstone Project]



## Dataset Cleaning

* This dataset contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. 
* Once every few days, Starbucks sends out an offer to users of the mobile app. 
* An offer can be merely an advertisement for a drink (informational offer) or an actual offer such as a discount or BOGO (buy one get one free).

We have three **JSON files**:

- ``profile.json``: Rewards program users.
- ``portfolio.json``: Offers sent during 30-day test period.
- ``transcript.json``: Event log.

## Data Cleaning 

- **Data cleaning** is a critically important step in any machine learning project.
- **Data cleaning** is a key step before any form of analysis can be made on it.
- **Data cleaning** not only refers to removing chunks of unnecessary data, but it’s also often associated with fixing incorrect information within the dataset and reducing duplicates.
- **Data cleaning** refers to the removal of unwanted data in the dataset before it’s fed into the model
- 

### **1. Libraries**
- Importing useful libraries 

In [1]:
import pandas as pd
import numpy as np
import math
import json
%matplotlib inline

import re
import datetime
from sklearn.preprocessing import LabelBinarizer


## 2. Reading the Dataset Files:
- Import the files using a module in pandas that reads JSON automatically.
- We are supposed to add a parameter lines set to True, since this is how our files are organized.


In [2]:
# read in the json files
portfolio = pd.read_json('portfolio.json', orient='records', lines=True)
profile = pd.read_json('profile.json', orient='records', lines=True)
transcript = pd.read_json('transcript.json', orient='records', lines=True)

## 3. Shape of the Dataset Files
- Using **.Shape()** function we will know the dimensions of each Dataset Files.
- The  **Shape** is a tuple that always gives dimensions of the array. 
- The **Shape** is a tuple that gives you an indication of the no. of dimensions in the array.
- Return a tuple representing the dimensionality of the DataFrame.

In [3]:
print("Portfolio Dimension = ",portfolio.shape,"\n")
print ("Profile Dimension = ",profile.shape,"\n")
print ("Transcript Dimension = ",transcript.shape,"\n")


Portfolio Dimension =  (10, 6) 

Profile Dimension =  (17000, 5) 

Transcript Dimension =  (306534, 4) 



## **4. Peek on the Three Tables**
- head() method is used to return top n (5 by default) rows of a data frame or series.
- In our case it wi return Ten rows of each Portfolio,Portfolio and Portfolio.

## **4.1. Portfolio Tables**

In [4]:
portfolio.head(5)


,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


## **4.2. Portfolio Table**

In [5]:
profile.head(5)

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [6]:
profile = profile.iloc[:, [1, 0] + list(range(2, profile.shape[1]))]

In [7]:
profile

,age,gender,id,became_member_on,income
0,118,None,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,55,F,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,118,None,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,75,F,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,118,None,a03223e636434f42ac4c3df47e8bac43,20170804,NaN
...,...,...,...,...,...
16995,45,F,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,54000.0
16996,61,M,2cb4f97358b841b9a9773a7aa05a9d77,20180713,72000.0
16997,49,M,01d26f638c274aa0b965d24cefe3183f,20170126,73000.0
16998,83,F,9dc1421481194dcd9400aec7c9ae6366,20160307,50000.0


## **4.3. Transcript Table**

In [8]:
transcript.head(5)

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


## **5. Dataset Cleaning**
- In this part, we will clean the data.
- The main aim of Data Cleaning is to identify and remove errors & duplicate data, in order to create a reliable dataset. 
- This improves the quality of the training data for analytics and enables accurate decision-making.
- So, our model will perform better and this will reflect on the accurcy.
- It is important to process data before use beacause Without quality data, it is hard to expect a correct output, performance and prediction.
- When we clean your data, all outdated or incorrect information is gone and leaving us with the highest quality information.


## **5.1. Portfolio File:**
- **Offers sent during 30-day test period (10 offers x 6 fields)**
- **reward:** *(numeric)* money awarded for the amount spent.
- **channels:** *(list)* web, email, mobile, social.
- **difficulty:** *(numeric)* minimum required spend to complete an offer.
- **duration:** *(numeric)* time for offer to be open, in days.
- **offer_type:** *(string)* bogo, discount, informational.
- **id:** *(string/hash)* offer id.

In [9]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


### **5.1.1. Portfoio Cleaning:**
- **Offers sent during 30-day test period (10 offers x 6 fields)**
-  we notice that there are no missing values.
- Column ``channels``;
     - Have multiple information in each row. 
     - We can make each channel's columns and input boolean values (0, 1).

- Column``id``:
   - column name change to 'offerid' for convenience when we merge the datasets.
   - Removing underscores in the column names.
- Transform column ``offertype`` that contain multiple type of offers that is: *`` bogo ``* ,*`` informational``*, *``discount``*  
   - *`` bogo ``* column
   - *`` informational``* column
   - *``discount``* column
   

#### - Checking for any missing values.

In [10]:
portfolio.isnull().sum()

reward        0
channels      0
difficulty    0
duration      0
offer_type    0
id            0
dtype: int64

#### -  Rename columns ``id `` to ``offerid`` to reduce confusion when we combine all tables.

In [11]:
portfolio.rename(columns = {'id':'offerid'}, inplace = True)

In [12]:
portfolio.columns = [re.sub('_','', name) for name in portfolio.columns]
print('Removing underscores in the column names Done.')

Removing underscores in the column names Done.


In [13]:
portfolio

,reward,channels,difficulty,duration,offertype,offerid
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


#### - As we see that column ``offertype`` that contain multiple type of offers that is: *`` bogo ``* ,*`` informational``*, *``discount``* ,  
#### We will use Pandas get_dummies method to transform categorical``offertype``column to multiple columns:
#### - *`` bogo ``* column.
#### - *`` informational``* column.
#### - *``discount``* column.

###### *Note:*
###### - get_dummies: Convert categorical variable into dummy/indicator variables Data of which to get dummy indicators.
###### - One Hot Encoding is a process in the data processing that is applied to categorical data, to convert it into a binary vector representation.

In [14]:
offertype_df = pd.get_dummies(portfolio['offertype'])
portfolio = pd.concat([portfolio, offertype_df], axis=1)
print('Transforming offer types Done.')

Transforming offer types Done.


In [15]:
portfolio

,reward,channels,difficulty,duration,offertype,offerid,bogo,discount,informational
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed,0,0,1
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4,0,1,0
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837,0,0,1
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5,0,1,0


#### - As we have seen that column ``channels`` contain multiple name *`` email ``* ,*`` mobile``*, *``social``* , *``web``* so, We Created dummy columns for the channels column.

In [16]:
temp = pd.get_dummies(portfolio.channels.apply(pd.Series).stack()).sum(level=0)
portfolio = pd.concat([portfolio, temp], axis=1, sort=False)

#### Dropping the Original Column ```channels``` 

In [17]:
portfolio.drop(columns='channels', inplace=True)
print('One-hot encoding channels.')

One-hot encoding channels.


In [18]:
portfolio

,reward,difficulty,duration,offertype,offerid,bogo,discount,informational,email,mobile,social,web
0,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,1,1,1,0
1,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1,1,1,1
2,0,0,4,informational,3f207df678b143eea3cee63160fa8bed,0,0,1,1,1,0,1
3,5,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1,1,0,1
4,5,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,1,0,0,1
5,3,7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0,1,1,1,1
6,2,10,10,discount,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,1,1,1,1
7,0,0,3,informational,5a8bc65990b245e5a138643cd4eb9837,0,0,1,1,1,1,0
8,5,5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0,1,1,1,1
9,2,10,7,discount,2906b810c7d4411798c6938adc9daaa5,0,1,0,1,1,0,1


In [19]:
portfolio = portfolio.drop(columns=['offertype'])
print(' portfolio Cleaning is completed.')

 portfolio Cleaning is completed.


In [20]:
portfolio
#portfolio.head()

,reward,difficulty,duration,offerid,bogo,discount,informational,email,mobile,social,web
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,1,1,1,0
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1,1,1,1
2,0,0,4,3f207df678b143eea3cee63160fa8bed,0,0,1,1,1,0,1
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1,1,0,1
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,1,0,0,1
5,3,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0,1,1,1,1
6,2,10,10,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,1,1,1,1
7,0,0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,1,1,1,1,0
8,5,5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0,1,1,1,1
9,2,10,7,2906b810c7d4411798c6938adc9daaa5,0,1,0,1,1,0,1


In [21]:
#portfolioc=portfolio.copy()
#portfolioc

In [22]:
#cols = list(portfolioc.columns)
cols = list(portfolio.columns)

In [23]:
a, b, c, d, e, f, g, h, x, y, z = cols.index('reward'), cols.index('difficulty'), cols.index('duration'), cols.index('offerid'),cols.index('bogo'), cols.index('discount'), cols.index('informational'), cols.index('email'), cols.index('mobile'), cols.index('social'), cols.index('web')

In [24]:
cols[a], cols[b], cols[c], cols[d] ,cols[e], cols[f], cols[g], cols[h],cols[x], cols[y], cols[z] = cols[b], cols[c], cols[d], cols[a] ,cols[e], cols[f], cols[g], cols[h],cols[x], cols[y], cols[z]
portfolio= portfolio[cols]

In [25]:
#cols[a], cols[b], cols[c], cols[d] ,cols[e], cols[f], cols[g], cols[h],cols[x], cols[y], cols[z] = cols[b], cols[c], cols[d], cols[a] ,cols[e], cols[f], cols[g], cols[h],cols[x], cols[y], cols[z]
#portfolioc = portfolioc[cols]

In [26]:
portfolio

,difficulty,duration,offerid,reward,bogo,discount,informational,email,mobile,social,web
0,10,7,ae264e3637204a6fb9bb56bc8210ddfd,10,1,0,0,1,1,1,0
1,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,10,1,0,0,1,1,1,1
2,0,4,3f207df678b143eea3cee63160fa8bed,0,0,0,1,1,1,0,1
3,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,1,0,0,1,1,0,1
4,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,0,1,0,1,0,0,1
5,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,3,0,1,0,1,1,1,1
6,10,10,fafdcd668e3743c1bb461111dcafc2a4,2,0,1,0,1,1,1,1
7,0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,0,1,1,1,1,0
8,5,5,f19421c1d4aa40978ebb69ca19b0e20d,5,1,0,0,1,1,1,1
9,10,7,2906b810c7d4411798c6938adc9daaa5,2,0,1,0,1,1,0,1


#### Portfolio Now is Cleaned and the Final Table is :
| reward | difficulty | duration | offerid | bogo | discount | informational | email | mobile | social | web |
|--------|------------|----------|---------|------|----------|---------------|-------|--------|--------|-----|

## **5.2. Profile File:**
- **Rewards program users (17000 users x 5 fields)**
- **gender:** *(categorical)* M, F, O, or null (note some entries contain 'O' for other rather than M or F)
- **age:** *(numeric)* age of the customer.
- **id:** *(string/hash)* customer's id.
- **became_member_on:** *(date)* format YYYYMMDD: date when customer created an app account.
- **income:** *(numeric)* customer's income.


In [27]:
profile

,age,gender,id,became_member_on,income
0,118,None,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,55,F,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,118,None,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,75,F,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,118,None,a03223e636434f42ac4c3df47e8bac43,20170804,NaN
...,...,...,...,...,...
16995,45,F,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,54000.0
16996,61,M,2cb4f97358b841b9a9773a7aa05a9d77,20180713,72000.0
16997,49,M,01d26f638c274aa0b965d24cefe3183f,20170126,73000.0
16998,83,F,9dc1421481194dcd9400aec7c9ae6366,20160307,50000.0


### **5.2.1. Profile Cleaning:**
- change ``id`` column name to ``customer_id``.
- Create ``age_group`` column, so we can see the distribution of age group in visualization section.
- Change ``became_member_on`` column dtypes as datetime, and extract year column.
- Change the values which have 118 value to 'NaN'  ``age`` columns.
- binarize the ``gender`` column.

#### Rename ``id `` column to ``customer_id `` column for more clearity of the columns names.

In [28]:
profile = profile.rename(columns={'id': 'customerid'})

In [29]:
print(profile.columns)

Index(['age', 'gender', 'customerid', 'became_member_on', 'income'], dtype='object')


#### As we're observing the missing values in the ``gender`` column is encoded as 118 in column ``age`` .
##### The ``age`` column all the [age = 118 ] have null value in mutiple columns ``income`` and ``gender``  we will Drop all the rows that have [age = 118] beacause it's safe to drop those rows as it null values and dropping them will make the dataset more simple and more informative.

In [30]:
profile.loc[(profile['age']==118) & (profile['income'].isnull()==True)].shape

(2175, 5)

In [31]:
profile.dropna(inplace=True)

In [32]:
print('The New shape of the profile data: ', profile.shape)

The New shape of the profile data:  (14825, 5)


#### - Remove customers with null income

In [33]:
profile = profile[profile['income'].isnull()==False]

###### -Filter the ``gender`` column 

In [34]:
profile = profile[profile['gender'] != 'O']
print('Other gender is filtered out.')

Other gender is filtered out.


##### - Reset the index 
##### reset_index will delete the index instead of inserting it back into the columns of the DataFrame. If you set drop = True , the current index will be deleted entirely and the numeric index will replace it.

In [35]:
profile.reset_index(drop=True)

,age,gender,customerid,became_member_on,income
0,55,F,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
1,75,F,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
2,68,M,e2127556f4f64592b11af22de27a7932,20180426,70000.0
3,65,M,389bc3fa690240e798340f5a15918d5c,20180209,53000.0
4,58,M,2eeac8d8feae4a8cad5a6af0499a211d,20171111,51000.0
...,...,...,...,...,...
14608,45,F,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,54000.0
14609,61,M,2cb4f97358b841b9a9773a7aa05a9d77,20180713,72000.0
14610,49,M,01d26f638c274aa0b965d24cefe3183f,20170126,73000.0
14611,83,F,9dc1421481194dcd9400aec7c9ae6366,20160307,50000.0


In [36]:
profile

,age,gender,customerid,became_member_on,income
1,55,F,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
3,75,F,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
5,68,M,e2127556f4f64592b11af22de27a7932,20180426,70000.0
8,65,M,389bc3fa690240e798340f5a15918d5c,20180209,53000.0
12,58,M,2eeac8d8feae4a8cad5a6af0499a211d,20171111,51000.0
...,...,...,...,...,...
16995,45,F,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,54000.0
16996,61,M,2cb4f97358b841b9a9773a7aa05a9d77,20180713,72000.0
16997,49,M,01d26f638c274aa0b965d24cefe3183f,20170126,73000.0
16998,83,F,9dc1421481194dcd9400aec7c9ae6366,20160307,50000.0


#### - Pre-processing The `` Date`` Column 
#### - Changing column ``became_member_on`` so it would be day,month, year ``DD,MM,YY``

In [37]:
profile['became_member_on'] = profile['became_member_on'].apply(lambda elem: datetime.datetime.strptime(str(elem), '%Y%m%d'))
profile['startyear'] = profile['became_member_on'].apply(lambda elem: elem.year)
print('Date column is pre-processed.')
    

Date column is pre-processed.


##### one-hot encode the start year using get_dummies which Convert categorical variable into dummy/indicator variables Data of which to get dummy indicators.

In [38]:
tmp = pd.get_dummies(profile['startyear'])   

##### binarize the gender column Label Binarizer is an class that accepts Categorical data as input and returns an Numpy array [0,1].

In [39]:
obj = LabelBinarizer()
profile['gender'] = obj.fit_transform(profile['gender'])
profile = pd.concat([profile, tmp], axis=1)
profile = profile.drop(columns=['startyear', 'became_member_on'])
print('Profile data is now cleaned.')

Profile data is now cleaned.


In [40]:
profile.head(5)

,age,gender,customerid,income,2013,2014,2015,2016,2017,2018
1,55,0,0610b486422d4921ae7d2bf64640c50b,112000.0,0,0,0,0,1,0
3,75,0,78afa995795e4d85b5d9ceeca43f5fef,100000.0,0,0,0,0,1,0
5,68,1,e2127556f4f64592b11af22de27a7932,70000.0,0,0,0,0,0,1
8,65,1,389bc3fa690240e798340f5a15918d5c,53000.0,0,0,0,0,0,1
12,58,1,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,0,0,0,0,1,0


## 5.3. Transcript File: 
- **Event log (306648 events x 4 fields)**
- **person:** *(string/hash)* customer id.
- **event:** *(string)* record description (offer received, offer viewed, transaction, offer completed).
- **value:** *(dictionary)* different values depending on event type.
- **offer id:** *(string/hash)* not associated with any "transaction".
- **amount:** *(numeric)* money spent in "transaction".
- **reward:** *(numeric)* money gained from "offer completed".
- **Time:** *(numeric)* hours after start of test.


In [41]:
transcript

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0
...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714


### *There are fours types of events:*

- **offer received:** when a given offer is sent to a customer;
- **offer viewed:** when the customer actually visualized the offer;
- **transaction:** any buy user made;
- **offer completed:** offer requirements were properly satisfied.

In [42]:
transcript.isna().any()

person    False
event     False
value     False
time      False
dtype: bool

In [43]:
transcript.event.unique()

array(['offer received', 'offer viewed', 'transaction', 'offer completed'],
      dtype=object)

#### First,  we need to check for any missing values in Transcript Tabel.

In [44]:
transcript.isnull().sum()

person    0
event     0
value     0
time      0
dtype: int64

##### function ```def construct_offer_transaction_data(data):``` process the transcript data to construct offer data and transaction data
- return: offer data dataframe and transaction dataframe.
- There are four types of events:
   - 'offer received'
   - 'offer viewed'
   - 'transaction'
   - 'offer completed'

In [45]:
def construct_offer_transaction_data(data):
    """
    This function process the transcript data to contruct offer data and transaction data.
    return: offer data dataframe and transaction dataframe
    """
    # Select customer offers
    obj = re.compile('^offer (?:received|viewed|completed)')

    is_offer = data['event'].apply(lambda elem: obj.match(elem) != None)

    offer_data = data[is_offer].copy()
    offer_data = offer_data.reset_index(drop=True)

    # Create an offerid column
    offer_data['offerid'] = offer_data['value'].apply(lambda elem: list(elem.values())[0])

    # Transform a column that describes a customer offer event
    obj = re.compile('^offer ([a-z]+$)')

    offer_data['event'] = offer_data['event'].apply(lambda elem: obj.match(elem).groups(1)[0])

    # One hot encode customer offer events
    event_df = pd.get_dummies(offer_data['event'])

    # Create a DataFrame that describes customer offer events
    offer_data = pd.concat([offer_data, event_df], axis=1)
    offer_data.drop(columns=['event', 'value'])
    
    #make transaction data
    transaction = data[is_offer == False]
    transaction = transaction.reset_index(drop=True)

    # Transform customer transaction event values
    transaction['amount'] = transaction['value'].apply(lambda elem: list(elem.values())[0])

    # Create a DataFrame that describes customer transactions
    transaction = transaction.drop(columns=['event', 'value'])
    
    return offer_data, transaction

### **5.3.1. Transcript Cleaning:**
- tramsfroms the transcript data.
- Changing person column name to customer_id
- Creating columns for different event type values - offerid, amount, and reward by extracting inf from value column.
- ``time`` column is expressed in hours, so it would be useful to calculate it in days, and thus comparable against the duration of the promotional offers.
- On-hot encode the relevant columns.
- return: offer_data dataframe and transaction dataframe.

#### This function process the transcript data to contruct offer data and transaction data and 
#### return: offer data dataframe and transaction dataframe

In [46]:
def make_offer_transaction_data(data):
    """
    This function process the transcript data to contruct offer data and transaction data.
    return: offer data dataframe and transaction dataframe
    """
    # Select customer offers
    obj = re.compile('^offer (?:received|viewed|completed)')

    is_offer = data['event'].apply(lambda elem: obj.match(elem) != None)

    offer_data = data[is_offer].copy()
    offer_data = offer_data.reset_index(drop=True)

    # Create an offerid column
    offer_data['offerid'] = offer_data['value'].apply(lambda elem: list(elem.values())[0])

    # Transform a column that describes a customer offer event
    obj = re.compile('^offer ([a-z]+$)')

    offer_data['event'] = offer_data['event'].apply(lambda elem: obj.match(elem).groups(1)[0])

    # One hot encode customer offer events
    event_df = pd.get_dummies(offer_data['event'])

    # Create a DataFrame that describes customer offer events
    offer_data = pd.concat([offer_data, event_df], axis=1)
    offer_data.drop(columns=['event', 'value'])
    
    #make transaction data
    transaction = data[is_offer == False]
    transaction = transaction.reset_index(drop=True)

    # Transform customer transaction event values
    transaction['amount'] = transaction['value'].apply(lambda elem: list(elem.values())[0])

    # Create a DataFrame that describes customer transactions
    transaction = transaction.drop(columns=['event', 'value'])
    
    return offer_data, transaction


#### This function cleans and tramsfroms the transcript data.
- Rename the columns to be more meaningful.
- Convert the number of hours into number of days for simplicity.
- On-hot encode the relevant columns.
- return: offer_data dataframe and transaction dataframe

In [47]:
#transcript = transcript.rename(columns={'person': 'customer_id'})
#transcript.loc[:, "time_in_days"] = transcript.loc[:, "time"] / 24
def clean_transcript_data(data):
    # rename the columns
    data.rename(columns = {'person':'customerid'}, inplace = True) 
    
    # remove those customer ids which are not in profile data
    data = data[data['customerid'].isin(profile['customerid'])]
    
    # convert hours to days for easy understanding
    #data['time'] /=24
    data.loc[:, "time"] =  data.loc[:, "time"] / 24
    data = data.rename(columns={'time': 'days'})
    
    
    offer_data, transaction_data = make_offer_transaction_data(data)
    
    return offer_data, transaction_data

offer_data, transaction_data = clean_transcript_data(transcript)


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [48]:
transaction_data.head()

,customerid,days,amount
0,02c083884c7d45b39cc68e1314fec56c,0.0,0.83
1,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0.0,34.56
2,54890f68699049c2a04d415abc25e717,0.0,13.23
3,b2f1cd155b864803ad8334cdf13c4bd2,0.0,19.51
4,fe97aa22dd3e48c8b143116a8403dd52,0.0,18.97


In [49]:
offer_data.head()

,customerid,event,value,days,offerid,completed,received,viewed
0,78afa995795e4d85b5d9ceeca43f5fef,received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,1,0
1,e2127556f4f64592b11af22de27a7932,received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0.0,2906b810c7d4411798c6938adc9daaa5,0,1,0
2,389bc3fa690240e798340f5a15918d5c,received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},0.0,f19421c1d4aa40978ebb69ca19b0e20d,0,1,0
3,2eeac8d8feae4a8cad5a6af0499a211d,received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},0.0,3f207df678b143eea3cee63160fa8bed,0,1,0
4,aa4862eba776480b8bb9c68455b8c2e1,received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0.0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0


#### The Dataset  provided by Starbucks is preprocessed (by cleaning rows with missed information) and converted into one table:

In [50]:
clean_data = []

# combining the data to construct meaning
customerid_list = offer_data['customerid'].unique()

for customerid in customerid_list:
    # select profile of the customer
    cur_customer = profile[profile['customerid']==customerid]
    
    # select the associated offers of that customer
    customer_offer_data = offer_data[offer_data['customerid']==customerid]
    customer_offer_data = customer_offer_data.drop(columns='customerid')
    customer_offer_data.reset_index(drop=True)
    
    # select transaction associated to that customer
    customer_transaction_data = transaction_data[transaction_data['customerid']==customerid]
    customer_transaction_data = customer_transaction_data.drop(columns='customerid')
    customer_transaction_data.reset_index(drop=True)
    
    event_type=['completed','received','viewed','value']
    offer_received = customer_offer_data[customer_offer_data['received']==1]
    offer_received = offer_received.drop(columns=event_type).reset_index(drop=True)
    
    offer_viewed = customer_offer_data[customer_offer_data['viewed']==1].drop(columns=event_type).reset_index(drop=True)
    
    offer_completed = customer_offer_data[customer_offer_data['completed']==1].drop(columns=event_type).reset_index(drop=True)

    rows=[]
    for idx in range(offer_received.shape[0]):
        cur_offer_id = offer_received.iloc[idx]['offerid']
        cur_offer = portfolio.loc[portfolio['offerid']==cur_offer_id]
        days = cur_offer['duration'].values[0]
        cur_offer_startime = offer_received.iloc[idx]['days']
        cur_offer_endtime = offer_received.iloc[idx]['days'] + days
        
        # select the transactions which falls in the validity period
        select_transaction = np.logical_and(customer_transaction_data['days']>=cur_offer_startime,
                                           customer_transaction_data['days']<=cur_offer_endtime)
        # select the offers viewed which falls in the validity period
        select_offer_viewed = np.logical_and(offer_viewed['days']>=cur_offer_startime,
                                            offer_viewed['days']<=cur_offer_endtime)
        # select the offers completed which falls in the validity period
        select_offer_completed = np.logical_and(offer_completed['days']>=cur_offer_startime,
                                               offer_completed['days']<=cur_offer_endtime)
        
        # determine whether the curent offer was successful
        cur_offer_successful = select_offer_completed.sum() > 0 and select_offer_viewed.sum()>0
        cur_offer_transaction = customer_transaction_data[select_transaction]
        
        # collect the information in dictionary format
        cur_row={
            'offerid': cur_offer_id,
            'customerid': customerid,
            'time':cur_offer_startime,
            'offersuccessful':int(cur_offer_successful),
            'totalamount':cur_offer_transaction['amount'].sum()
        }
       
        cur_row.update(cur_offer.iloc[0,1:].to_dict())

        cur_row.update(cur_customer.iloc[0,1:].to_dict())

        rows.append(cur_row)
    clean_data.extend(rows)
    
    
clean_data = pd.DataFrame(clean_data)
clean_data.head()

    

,offerid,customerid,time,offersuccessful,totalamount,duration,reward,bogo,discount,informational,...,social,web,gender,income,2013,2014,2015,2016,2017,2018
0,9b98b8c7a33c4b65b9aebfe6a799e6d9,78afa995795e4d85b5d9ceeca43f5fef,0.0,1,37.67,7,5,1,0,0,...,0,1,0,100000.0,0,0,0,0,1,0
1,5a8bc65990b245e5a138643cd4eb9837,78afa995795e4d85b5d9ceeca43f5fef,7.0,0,49.39,3,0,0,0,1,...,1,0,0,100000.0,0,0,0,0,1,0
2,ae264e3637204a6fb9bb56bc8210ddfd,78afa995795e4d85b5d9ceeca43f5fef,17.0,1,48.28,7,10,1,0,0,...,1,0,0,100000.0,0,0,0,0,1,0
3,f19421c1d4aa40978ebb69ca19b0e20d,78afa995795e4d85b5d9ceeca43f5fef,21.0,1,48.28,5,5,1,0,0,...,1,1,0,100000.0,0,0,0,0,1,0
4,2906b810c7d4411798c6938adc9daaa5,e2127556f4f64592b11af22de27a7932,0.0,0,0.00,7,2,0,1,0,...,0,1,1,70000.0,0,0,0,0,0,1


In [51]:
clean_data.columns

Index([        'offerid',      'customerid',            'time',
       'offersuccessful',     'totalamount',        'duration',
                'reward',            'bogo',        'discount',
         'informational',           'email',          'mobile',
                'social',             'web',          'gender',
                'income',              2013,              2014,
                    2015,              2016,              2017,
                    2018],
      dtype='object')

In [52]:
clean_data = clean_data.sort_values('time')

#### Saving the Cleadned Dataset int CSV file for further using

In [53]:
clean_data.to_csv('Data_cleaned.csv')


In [54]:
pd.read_csv('Data_cleaned.csv')

,Unnamed: 0,offerid,customerid,time,offersuccessful,totalamount,duration,reward,bogo,discount,...,social,web,gender,income,2013,2014,2015,2016,2017,2018
0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,78afa995795e4d85b5d9ceeca43f5fef,0.0,1,37.67,7,5,1,0,...,0,1,0,100000.0,0,0,0,0,1,0
1,11262,4d5c57ea9a6940dd891ad53e9dbe8da0,ff7cb44e72db4112b270560686f97a23,0.0,1,48.31,5,10,1,0,...,1,1,0,39000.0,0,0,1,0,0,0
2,35019,5a8bc65990b245e5a138643cd4eb9837,97b6993c232946d3b6b9f90530ff8073,0.0,0,23.43,3,0,0,0,...,1,0,1,52000.0,0,0,0,0,1,0
3,35016,9b98b8c7a33c4b65b9aebfe6a799e6d9,81848348d5584aef9e7374a07ebe6ea1,0.0,0,52.24,7,5,1,0,...,0,1,0,118000.0,0,0,0,1,0,0
4,11266,fafdcd668e3743c1bb461111dcafc2a4,28f9666945804ab0bfc63f3ec6ae9af1,0.0,0,5.12,10,2,0,1,...,1,1,0,44000.0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65580,29227,0b1e1539f2cc45b7b9fa7c272da2e1d7,201969e0f3a7480eba5a291687c45cb9,24.0,0,0.00,10,5,0,1,...,0,1,0,36000.0,0,0,1,0,0,0
65581,51545,0b1e1539f2cc45b7b9fa7c272da2e1d7,cbebad9819494b479dbf0c1207b4c0c5,24.0,1,31.68,10,5,0,1,...,0,1,1,70000.0,0,0,0,0,1,0
65582,13958,0b1e1539f2cc45b7b9fa7c272da2e1d7,c727c0939de24b92a35d4fd7c9fec518,24.0,0,0.00,10,5,0,1,...,0,1,1,56000.0,0,0,0,0,0,1
65583,3570,0b1e1539f2cc45b7b9fa7c272da2e1d7,727a658b33b34d39ab14fc30bcd9eed9,24.0,0,0.00,10,5,0,1,...,0,1,1,95000.0,0,0,0,0,1,0
